# Methane Emission Locator Tool

## Overview
This Jupyter notebook will help you track down the specific location of a methane leak at a landfill. Please run the code in order and follow any instructions given. Should you have any problems, please refer to the relevant section in the user manual. 

This tool works using the followig formula: 

            frac_change = ((c * no_emission_data - active_emission_data) / active_emission_data)  

Where:
- 'c' is a scaling factor that adjusts for scene-wide changes in brightness between satellite passes. 
- 'no_emission_data' is a dataset WITHOUT an emission.
- 'active_emission_data' is a dataset WITH an emission. 

The dates for the 'active_emission_data' and 'no_emission_data' can be determined by using the Sentinel 5P Time Series, or Sentinel 5P CH4 Map tools.

<u>WARNING - If you wish to run this tool twice (for different dates) then the kernel will need to be reset each time.</u> 

The section below imports the packages needed to run the script.

In [ ]:
import folium
import pandas as pd
import matplotlib.pyplot as plt
import openeo
import rasterio
import numpy as np
import requests
from sklearn.linear_model import LinearRegression

## Connect to OpenEO

The code below establishes a connection with the Copernicus openEO platform which provides a wide variety of earth observation datasets

- If this does not read as 'Authorised successfully' or 'Authenticated using refresh token', then please ensure that you have completed the setup steps as outlined in section 2.6 of the user guide. 

- If you have followed the steps in section 2.6 correctly and the problem persists, please look at https://dataspace.copernicus.eu/news for any information about service interruptions. 

- If there is no news of service problems you can raise a ticket here: https://helpcenter.dataspace.copernicus.eu/hc/en-gb/requests/new

In [ ]:
connection = openeo.connect(url="openeo.dataspace.copernicus.eu")
connection.authenticate_oidc()

## View list of landfills. 

Each landfill has been asigned an id number. This number can be found in the list produced by the following code. To look at a chosen landfill, you will need to input that number at various stages in the code. Run the code below to the the list. The west, south, east and north columns are latitude and longitude cordinates to establish a box around each landfill for analysis.  

In [ ]:
landfill_csv = pd.read_csv(r'C:\GIS_Course\EGM722\Project\Data\PreZero_Landfill_Bounding.csv')

landfill_csv

# Available dates for the analysis. 

Unlike Sentinel 5P, Sentinel 2 only provides data aproximately once every 3 days. Because of this, not every date you can enter into this tool is valid. The code below will tell you what dates are available to use for the landfill of your choice. 

The two parameters you need to modify before running the code are: 
- landfill_id = 23 (change this to your chosen landfill) 
- temporal_extent = ["2023-01-31", "2023-03-12"] (change this to your chosen date range using "YYYY-MM-DD" format)

Once you have done this run the code and the available dates should appear below in a matter of a few seconds. 

In [ ]:
def get_spatial_extent(landfill_id):
    landfill = landfill_csv[landfill_csv['id'] == landfill_id].iloc[0]
    return {
        "west": landfill['west'],
        "south": landfill['south'],
        "east": landfill['east'],
        "north": landfill['north']
    }

def fetch_available_dates(landfill_id, temporal_extent):
    spatial_extent = get_spatial_extent(landfill_id)
    catalog_url = f"https://catalogue.dataspace.copernicus.eu/resto/api/collections/Sentinel2/search.json?box={spatial_extent['west']}%2C{spatial_extent['south']}%2C{spatial_extent['east']}%2C{spatial_extent['north']}&sortParam=startDate&sortOrder=ascending&page=1&maxRecords=1000&status=ONLINE&dataset=ESA-DATASET&productType=L2A&startDate={temporal_extent[0]}T00%3A00%3A00Z&completionDate={temporal_extent[1]}T00%3A00%3A00Z"
    response = requests.get(catalog_url)
    response.raise_for_status()
    catalog = response.json()
    dates = [date.split('T')[0] for date in map(lambda x: x['properties']['startDate'], catalog['features'])]
    return dates

# Please enter your perameters here.
landfill_id = 23  # Specify the landfill ID.
temporal_extent = ["2023-01-31", "2023-03-12"]  # Specify the the date range you want to check for available data.

available_dates = fetch_available_dates(landfill_id, temporal_extent)
print("Available dates:", available_dates)

## Choosing the Active Emission Date

As mentioned in the overview, an active emission date must be chosen from one of the available datasets. 

Like before, the two parameters you need to modify before running the code are:

- landfill_id = 23 (change this to your chosen landfill)
- temporal_extent = ["2023-02-25", "2023-02-25"] (change this to your chosen date range using "YYYY-MM-DD" format.) 

Please note that the temporal extent dates <u>MUST BE IDENTICAL</u> because we are only choosing a single date.

If you recieve an error message of 'NoDataAvailable' then please check the list of available data above and try again.


In [ ]:
def active_emission(landfill_id, temporal_extent):
    landfill = landfill_csv[landfill_csv['id'] == landfill_id].iloc[0]

    active_emission = connection.load_collection(
        "SENTINEL2_L2A",
        temporal_extent=temporal_extent,
        spatial_extent={
            "west": landfill['west'],
            "south": landfill['south'],
            "east": landfill['east'],
            "north": landfill['north']
        },
        bands=["B12"],
    )
    active_emission.download("Sentinel-2_active_emission.GTiff")

# Enter perameters for the active emission day
landfill_id = 23  # Specify the landfill ID
temporal_extent = ["2023-02-25", "2023-02-25"]

active_emission(landfill_id, temporal_extent)

## Choosing the No Emission Date

Next we choose the no emission date using the same process. 

The two parameters you need to modify before running the code are:

- landfill_id = 23 (change this to your chosen landfill)
- temporal_extent = ["2023-02-25", "2023-02-25"] (change this to your chosen date range using "YYYY-MM-DD" format.) 

The temporal extent dates <u>MUST BE IDENTICAL</u>

If you recieve an error message of 'NoDataAvailable' then please check the list of available data above and try again.


In [ ]:
def no_emission(landfill_id, temporal_extent):
    landfill = landfill_csv[landfill_csv['id'] == landfill_id].iloc[0]

    no_emission_collection = connection.load_collection(
        "SENTINEL2_L2A",
        temporal_extent=temporal_extent,
        spatial_extent={
            "west": landfill['west'],
            "south": landfill['south'],
            "east": landfill['east'],
            "north": landfill['north']
        },
        bands=["B12"],
    )
    no_emission_collection.download("Sentinel-2_no_emission.GTiff")

# Enter parameters for the no emission day
landfill_id = 23  # Specify the landfill ID
temporal_extent = ["2023-02-10", "2023-02-10"]

no_emission(landfill_id, temporal_extent)

## Choosing A Background Satelite Image

This section helps with locating the source of the emission at the landfill by displaying a true colour satelite image of the landfill that the data will be superimposed over. I recommend choosing the same date as your active emission. 

Once again, the two parameters you need to modify before running the code are:

- landfill_id = 23 (change this to your chosen landfill)
- temporal_extent = ["2023-02-25", "2023-02-25"] (change this to your chosen date range using "YYYY-MM-DD" format.)

The temporal extent dates <u>MUST BE IDENTICAL</u>

If you recieve an error message of 'NoDataAvailable' then please check the list of available data above and try again.

In [ ]:
def truecolour_image(landfill_id, temporal_extent):
    landfill = landfill_csv[landfill_csv['id'] == landfill_id].iloc[0]

    truecolour_image_collection = connection.load_collection(
        "SENTINEL2_L2A",
        temporal_extent=temporal_extent,
        spatial_extent={
            "west": landfill['west'],
            "south": landfill['south'],
            "east": landfill['east'],
            "north": landfill['north']
        },
        bands=["B02", "B03", "B04"],
    )
    truecolour_image_collection.download("Sentinel-2_truecolour.GTiff")

# Enter parameters for the no emission day
landfill_id = 23  # Specify the landfill ID
temporal_extent = ["2023-02-22", "2023-02-22"]

truecolour_image(landfill_id, temporal_extent)

## Running the Analysis
Here, no modification by you is needed. The code below runs the analysis.  Provided all the variables above have been run correctly, this next section should take moments to complete. 

In [ ]:
def compute_correction_factor(active_emission_data, no_emission_data):
    """
    Compute a correction factor c via linear regression
    with the following equation: active_emission_data = c*no_emission_data
    """
    # get flattened arrays for regression
    y = active_emission_data.flatten()
    x = no_emission_data.flatten()

    # replace NaN values with 0
    y = np.nan_to_num(y)
    x = np.nan_to_num(x)

    assert y.shape == x.shape, "Shapes of two provided arrays are incompatible!"

    # fit linear model using least squares regression
    x = x[:, np.newaxis]
    c, _, _, _ = np.linalg.lstsq(x, y, rcond=None)

    return c[0]

def compute_corrected_fractional_reflectance_change(active_emission_data, no_emission_data):
    """
    Compute corrected fractional reflectance change between base and monitoring scene
    """
    # get correction factor
    c = compute_correction_factor(active_emission_data, no_emission_data)

    # get corrected fractional reflectance change
    frac_change = ((c * no_emission_data - active_emission_data) / active_emission_data)

    return frac_change

# Open raster files
with rasterio.open(r"C:\GIS_Course\EGM722\Project\Sentinel-2_active_emission.GTiff") as active_emission_file:
    active_emission_data = active_emission_file.read(1)

with rasterio.open(r"C:\GIS_Course\EGM722\Project\Sentinel-2_no_emission.GTiff") as no_emission_file:
    no_emission_data = no_emission_file.read(1)

# Compute corrected fractional reflectance change
frac_change = compute_corrected_fractional_reflectance_change(active_emission_data, no_emission_data)

## Viewing the data. 

This section of code can be run to produce the map. There is one piece of code that can be altered if needed. 

- brightness_factor = 0.05 (occasionally the true colour satelite image can be too bright or too dark. You can change this number to fix it)

In [ ]:
# Open the and load the truecolour satelite image file and define which band is which.
fp = 'Sentinel-2_truecolour.GTiff'
img = rasterio.open(fp)
blue = img.read(1)
green = img.read(2)
red = img.read(3)

# Change this number up or down if the satelite background image is too dark or bright. 
brightness_factor = 0.05 
blue = np.clip(blue * brightness_factor, 0, 255)
green = np.clip(green * brightness_factor, 0, 255)
red = np.clip(red * brightness_factor, 0, 255)

# stack the blue, green and red bands to make a colour image. 
rgb = np.dstack((red, green, blue))
rgb = rgb / rgb.max()

#loading the methane data so it displays on top of the satelite image. 

# Create a masked array where values below 0.01 are masked
masked_frac_change = np.ma.masked_less_equal(frac_change, 0.01)

# Create a new figure
plt.figure(figsize=(10, 8))

# Display the RGB image
plt.imshow(rgb)

# Define the range of your colormap
vmin = 1
vmax = 10

# Display the masked_frac_change data on top of the RGB image
# Use the viridis colormap and set alpha to 0.5 for semi-transparency
plt.imshow(masked_frac_change, cmap='plasma', alpha=1, vmin=vmin, vmax=vmax)

# Add a colorbar and labels
cbar = plt.colorbar(label='Corrected Fractional Reflectance Change', shrink=0.7)
plt.title('Corrected Fractional Reflectance Change')

# Define the resolution of your image (in meters per pixel)
resolution = 20  # replace this with your actual resolution

# Get the dimensions of the image
height, width, _ = rgb.shape

# Create arrays representing the x and y coordinates in meters
x = np.arange(0, width * resolution, 1000)
y = np.arange(0, height * resolution, 1000)

# Set the x and y ticks and labels
plt.xticks(x / resolution, x)
plt.yticks(y / resolution, y)
plt.xlabel('X (meters)')
plt.ylabel('Y (meters)')

# Add grid lines
plt.grid(color='gray', linestyle='-', linewidth=0.5)

# Show the plot
plt.show()

## Interpreting this map

- The map above will show methane clouds in blue, purple and pink.
- Methane sources should show as bright orange and yellow points. 